In [ ]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver

# (201105) 추가한 모듈, BeautifulSoup4 사용 추가
from bs4 import BeautifulSoup

# lxml
from lxml import html

# pandas
import pandas as pd

# requests, json, re
import requests
import json

In [ ]:
# 유튜브 정식 카테고리 k, v dictionary
youtube_category_dict = {
    'Film & Animation' : 1,
    'Autos & Vehicles' : 2,
    'Music' : 10,
    'Pets & Animals' : 15,
    'Sports' : 17,
    'Short Movies' : 18,
    'Travel & Events' : 19,
    'Gaming' : 20,
    'Videoblogging' : 21,
    'People & Blogs' : 22,
    'Comedy' : 23,
    'Entertainment' : 24,
    'News & Politics' : 25,
    'Howto & Style' : 26,
    'Education' : 27,
    'Science & Technology' : 28,
    'Movies' : 30,
    'Anime/Animation' : 31,
    'Action/Adventure' : 32,
    'Classics' : 33,
    'Documentary' : 35,
    'Drama' : 36,
    'Family' : 37,
    'Foreign' : 38,
    'Horror' : 39,
    'Sci-Fi/Fantasy' : 40,
    'Thriller' : 41,
    'Shorts' : 42,
    'Shows' : 43,
    'Trailers' :44
}

# 샘플 키워드 리스트
sports_list = ['한국 프로야구', '해외축구', '바르셀로나', '배구', '손흥민']
movie_list = ['영화리뷰', '영화 명장면', '명대사', '한국 영화']
kids_list = ['뽀로로', '헤이지니', '카봇', '보람튜브', '핑크퐁', '공룡']
music_list = ['인기 음악', 'k-pop', '팝송', '카페에서 듣기좋은 노래', '호른', '클래식', '째즈']
comedy_list = ['런닝맨', '1박2일 다시보기', '무한도전', '놀면 뭐하니', 'tvN']
game_list = ['롤', '피파', 'fps', '배그', '서든어택', '어몽어스']
politics_list = ['바이든', '트럼프', '대통령', '중국']
cook_list = ['백종원', '자취음식', '편의점 음식만들기', '몸에 좋은 음식']
animal_list = ['고양이 영상', '비글', '해수어', '상어']

# 모든 키워드를 다 합친 리스트
all_search_keyword_list = sports_list + movie_list + kids_list + music_list + comedy_list + game_list + politics_list + cook_list + animal_list

print("All keywords list: ", all_search_keyword_list)

# video_id_list
# sports_video_id_list = []
# movie_video_id_list = []
# kids_video_id_list = []
# music_video_id_list = []
# comedy_video_id_list = []
# game_video_id_list = []
# politics_video_id_list = []
# cook_video_id_list = []
# animal_video_id_list = []

# 샘플 키워드 딕셔너리
# search_dict = {
#     'Sports' : [sports_list, sports_video_id_list]
#     'Movie' : [movie_list, movie_video_id_list]
#     'Kids' : [kids_list, kids_video_id_list]
#     'Music' : [music_list, music_video_id_list]
#     'Comedy' : [comedy_list, comedy_video_id_list]
#     'Game' : [game_list, game_video_id_list]
#     'Politics' : [politics_list, politics_video_id_list]
#     'Cook' : [cook_list, cook_video_id_list]
#     'Animal' : [animal_list, animal_video_id_list]
# }

In [ ]:
# 샘플 키워드 리스트 #2

In [ ]:
# 검색어 결과별로 비디오 id를 모아주는 함수 정의
# 리턴은 dictionary
def get_search_keyword_scrap_video_id(search_keyword_list):
    # 최종적으로 리턴할 dictionary 정의
    youtube_id_dict = {}

    for word in search_keyword_list:
        # youtube_id_list
        youtube_id_list = []
        
        # webdriver define
        driver_path = './chromedriver'
        driver = webdriver.Chrome(driver_path)
        driver.implicitly_wait(30) # or bigger second
        
        # 열고자 하는 search url 정의
        url = "https://www.youtube.com/results?search_query=" + word
        print("full url: ", url)

        # 해당 url로 창 열기
        driver.get(url)
        time.sleep(5)

        # 첫 동영상 시작 시 시작번호(index = 1)
        index = 1
        SCROLL_PAUSE_TIME = 2

        # infinite scroll
        while True:
            try:
                youtube_video_id_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer[1]/div[3]/ytd-video-renderer['+str(index)+']/div[1]/div/div[1]/div/h3/a'

                # 해당 페이지의 8개의 영상을 모두 확인했다면 그다음으로 가기위해 스크롤
                if index % 4 == 0:
                    # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    # driver.execute_script("window.scrollBy(0, 1080);")
                    # time.sleep(SCROLL_PAUSE_TIME)
                    last_height = driver.execute_script("return document.body.scrollHeight")

                    while True:
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(SCROLL_PAUSE_TIME)

                        new_height = driver.execute_script("return document.body.scrollHeight")

                        if new_height == last_height:
                            break

                        last_height = new_height


                youtube_title_link = driver.find_element_by_xpath(youtube_video_id_xpath)
                youtube_id = youtube_title_link.get_attribute('href')
                youtube_id = youtube_id.split("v=")[1].split("&")[0]
                youtube_id_list.append(youtube_id)

                # 리스트에 들어간 데이터 중간 확인
                print("index_no: ", index)
                print("youtube_id: ", youtube_id)

                index += 1

            except Exception as e:
                print()
                print("Exception: ", e)
                break
        # dictionary에 검색 키워드 별 리스트 추가
        youtube_id_dict[word] = youtube_id_list
        # youtube_id 수집이 끝난 브라우저 닫기        
        driver.close()
    
    return youtube_id_dict

In [ ]:
youtube_id_dict = get_search_keyword_scrap_video_id(all_search_keyword_list)
print("youtube_id_dict: ", youtube_id_dict)

In [ ]:
# youtube_id_dict
youtube_id_dict = {'한국 프로야구': ['doorlUyE9pI', 'L0Y38pQbEeQ', 'VjwJSj0AW4g', 'A5vCPtjOe8Q', 'Zeja7fC1Okc', '7ARbpqOgHM8', 'WC2MB_lZ0HQ', 'ugsFwK74KDU', 'FeqhJVzlE-4', 'B-ZFW6-FBJ8', 'kH-ht4CyUKg', 'kQxq3PNE0gw', 'leBGyyECGPk', '7eYu507yrQ8', '-P68flOonvE', 'FCEPBiUPPho', 'pp0a2SbPwRA', 'VFufiK95fGU', 'De58PXoH4IU', 'wVo7-0zDNps'], '해외축구': ['zP-gjvYibTM', 'DWwbxsVwPEs', 'jKJkhkU_Uy0', 'TI5LZt1MzkA', 'nxGNyWF2ASQ', 'XmDrZ_2Kg2U', 'Gf9t-ySPfao', 'BdHFW-U6_Pw', 'HuXhzWuRnos', 'VpEre3ksO5w', 'lchK_-AqmaQ', '5caHF8J62G4', 'epxwH4xMq9Q', 'N-gP5KyrWSU', '8FuIAt0Cz5M', 'LKJwO9PRuBg', 'qUTqVD8pMpI', 'b_y5cIw8toQ', '3Z0N2jTS3_k'], '바르셀로나': ['iwt-dKlsyc8', 'Ww2f6hcbgPo', 'DWwbxsVwPEs', '1R6eqSmlyKs', 'QBm8vthloeQ', '92oRB3yYSi4', 'V_1SvHgRQ9E', 'AQ2p8hxX90Y', 'LKjkugAHCmY', 'xChIWLtcT6Q', '2d6JJDZCIvc', 'TOC126WWIgc', 'xpHO-0JpP6w', 'b2NVhLUTLdM', 'j8onFrjLUdA', 'PtfS4o9o_gE', 'pkg__2CzXSk', 'omHQiFoD2JU', 'aHwfftlTK2c', 'Fw5AJryeTB4'], '배구': ['hDjlMQwI3n8', 'qSVzA_su0_Y', 'XE9WwgCZVic', 't4rmTfHRe58', 'zJBhOTezumM', '2GpNMDhpXPg', 'VT6lZWcQrQ8', 'I4nmh6PFF0Q', '6EJhGux0_g0', 'H_GrMKiRPLI', 'lpwR7eJS6OA', 'rOhAG74chwA', '_5ERlIGgWtA', 'ceS72R7gRAo', 'Szozl8dYPR8', 'xbJCrxfXndI', 'tsyM11aPQj4', 'SFRlCcDqZJY', '8f7c28DmNDY', 'RQRVAxBNsWg'], '손흥민': ['kd_YLeWRltY', 'XZzE7jO51_M', 'y7Ywwm8bvMA', 'UgrU6pDCJRU', 'omcGbZ5RbzI', 'F5b8sr4BKIg', 'iShyDckElHY', 'ATo-kwX0sJc', 'nPCeN8khVDA', 'Ka_vHYMOjlc', 'iQ1oi_p7ZCg', '7pwRhZnA8tk', 'dpyHS7vMNBA', 'rRF8v0v6m-o', 'na0xf3J0v4w', 'JZw1jooO4HA', 'GU3qCbMCEgI', 'ahQobPP5bF0', 'Icxcxeb8Yxg', 'JHp-ooo75C4'], '영화리뷰': ['A-Xu6CV-eRY', '0nHoVO_5Ius', 'fnCGDkR3t-A', 'cG2U9rFSKO4', 'KEQNpEOC660', 'j_-YWwVncQQ', 'AA0Z8yROvYg', 'CLveKMZ5R9E', 'x26o2y0ytEM', 'sxNkEbbDAbo', '1nY-8ZEngx0', 'WT9sN4hkNsI', 'to0yQ-8q-as', 'M54Z50Bcud8', 'bAdFEaLVaa8', 'aNVAxFANHlo', 'vgY--6eJj7s', 'ITPU2bNc4ts', 'kSDZ6KjgkAE', 'FkFkQFQkF84'], '영화 명장면': ['hdU_5M9U9tk', 'ojttyD8aMR0', '12d5V0lzdYg', 'u1VezOU7bt4', 'nUxfxGYjbO4', 'hd6bGMJ-Lwk', 'l6GGHe2hgtk', 'UvIEBsJnQyk', 'M6fa1SWeya4', 'um3c0hMzz5M', 'g3XqWtn681Y', 'jS5-1KUapuU', 'gb2s1eF_EGg', '7wWHZYS7JV4', 'VVGCzsm9r4E', '1gZElKODBBc', 'Joa8f3IN4dQ', '1N_3Du-JBIo', 'bxZXN3zcE5I', '_EWIe0PjQ6g'], '명대사': ['ojttyD8aMR0', 'hdU_5M9U9tk', '0ZHqB7Fplu0', 'goGst1xKBQ4', 'LJO7-pN9hAk', 'EwERszcjwaE', 'F4SUPzLdbu0', 'zm8Ts0XoErg', 'y58EHjtKnBA', 'Q9oat_q3UqQ', '9fCqGqqrNVk', 'SQfv__xer3A', 'OQVzaW0oBgE', 'XJkG4Kiykm8', 'pYCtl0CEv5w', 'B2mOlO1ERtw', '_ZWKRJY5Ghs', '8nRNGweY46g', 'R5NWIGAhHYA', 'dxusUW9_xZE'], '한국 영화': [], '뽀로로': ['xmHdqnDfqg4', 'xlmPbjAcRXQ', 'xCMPnwDkfDI', 'mJfrwP4wufM', 'sZkRT09tXEc', '_ee83gRY5qo', 'fDjsD8atx1U', 'Idc7hWtOeL0', 'ed56y-G4FcI', 'FfC8ZWAqV5Q', '02V4I_73FcE', '65ejhvVjViI', 'TtAsS3Sjq64', 'ZHSQTVB8zOI', 'g9kQEPDN5PU', 'XFUi4KTlkgA', 'UG4Z1edn8bQ', '-O9hpaEK_-Y', 'rCUXGWk3DHg'], '헤이지니': ['Ud7appVhiC0', 'VNMj3dgzMHw', 'pi82H6eZRSU', 'AKT2Z_cE5t0', 'olLFXLKEzvc', 'qX6odkUJ1lg', 'alQaHRb3HDk', 'xQfqaTTD1JY', 'pN4WmaIQQBU', 'PgcQ81tRV4w', '8zqrI3za7LM', 'a6p_TWoeGH8', '9Ks-DP_wp1s', 'MOMEssNSA8o', 'a-rysSCYT9w', 'IGCyavURxMg', 'Vq7jlQ0xaks', 'CN9hMZLphUU', 'OE-S7rEBXwk'], '카봇': ['fMRARhSQILg', '1qP9N175okQ', '2enS5vTUsWw', '3BEUwnqzQnY', '1WzYvNTR0Zk', 'mQfCs7KIjXs', 'tdYxvPfVyGY', 'BS28qiDPRWI', 'cfjTExuA_nM', 'PjE-4QdKufY', 'OJeRcK1Ml3k', 'zglQKs1adV8', 'xdJ4WcBh4Vg', 'kPL0gXwwwjE', 'al6v7SU3NNg', 'uk-NRDRiZaA', 'bHJ1SEarxtU', 'VXVmF1vnnFw', '0Qmt9bLbBOg'], '보람튜브': ['bzQIbntYh7c', 'X2Zmf3c0ccA', 'MqQUUdxrzAQ', 'NPTTJqcPhXU', 'PZYgwhJB7Oo', 'xEpL0_JjpHw', 'ht42dD1erJk', 'L__85-J9NXs', 'dumlv30AHJU', '3Xun01p1oZI', 'UpDo2vfusGw', 'hWdm8cHd7LU', 'YFe_L83o38w', '5Fw4eeh8Qys', 'uawQagHG5js', 'iQiGeK2vFpo', '4AC42XrAuoo', 'sC2AjRB7kWU'], '핑크퐁': ['WEHld_90DlM', 'dBD54EZIrZo', 'dkzNqzIC2-k', '_ELRHM_Fc14', 'kw_zTVuatk4', 'Z6bVY6Q87xY', 'hvTq3bLY2N8', 'rwnqNUALVbs', 'oqQyELG6K8Y', '9XTPX-t1TqM', '0rCMJkLh9VA', '6bbMFMRei2I', 'cq_JzIjis40', 'EeVRIya_pfA', 'wVlP86rgSQM', 'tTIw6PVxy5c', 'hg_HM447sOU', '8_7BZHfSmfM'], '공룡': ['JLFtlXPuFaw', '7IadsIPjO9I', 'v2tsHXQI6d0', 'p_Fj6KmgXX4', 'IRZF0IZ6BhI', 'vvAly7LI23U', 'JAYa0ffVVQY', 'm40Dt_TZFBI', 'IUAjhm7oE9c', '-FqABXx2t4o', '18N7NHp9yS4', 'iz2Pd_eP7Hw', 'Qj45k7hCIwI', 'xcFaoRPFT1A', 'f2JwkdOlBVU', 'RnO-GXbajww', '1wqQJpXBL_8', 'H9WA-pTdFOw', 'L0qoyEhwleU'], '인기 음악': ['zjGzPiYGhhY', 'khu6-Cdn8E4', '848a1kMwRpE', '_BVqb9vWZ_k', 'wMSh6wFNtuo', 'Rum2524qZPU', 'B8FyB1vf5lY', 'YyeylfmSVP8', 'vpbR3NEJmJM', 'Z2_ygVUxUcc', 'USgbeoynpw4', '0lI5xI_gIIs', 'u-E92ep1BDg', '-kkyXf_D_TE', 'yJuLOl1V-lQ', 'iHfqD2RYT2I', 'b5rrGmpZlDY', 'JzziwpDrigs', 'N5Nq3xBs5U8'], 'k-pop': ['rLxLVeN7sgc', 'GWFfWbDw5Q8', 'ZNV-pnhHdEs', '0nxsS4B85E8', 'ZeerrnuLi5E', '7c4gAOZ1JAw', 'qM2NJNrJDQI', 'zWiVhjmM9Hs', 'dH8e3Q_ExWk', 'T56iHifXrsY', 'h9TmtANeOmY', 'kOzoFmkXxZE', 'bgVixCyg7fA', 'Z3D8bN0UH_8', 'CM4CkVFmTds', 'UOxkGD8qRB4', '446ZxblW6pU', 'elPoQ_M4gaA', '0QbVt7N9FNo'], '팝송': ['_kd8bp0pp8U', 'HMfDUdNVjcw', 'NnSAsgkTgtY', 'mxJGDa7ThbE'], '카페에서 듣기좋은 노래': ['SUEbgBbJ1BQ', 'e9AkF2S1BR8', 'VcIs0Lvbsj4', 'SORD03t7nlo'], '호른': ['lNuJVfe-t3o', '8NdzRGJ4xJo', 'K10vpdVZ58Q', '4nU1Ny_Wplw', '6AVa7gU0v7E', 'RNo5l8iMRVw', '5WN20BcJsgw', 'fhT9f0ZKIRg', 'QB74ZZJk_bk', 'KKI51KIGvVw', 'CgNJSfwxiYw', '5fro7jmT3_M', 'MCH_MDdVEfM', 'iyEhtV-OSjw', 'TuZ3-O1EUIc', 'xAJQpZT5Q2g', 'M3OFiQtRI_A', 'SIoBIxKGL-Y', 'EY11eklYA5s'], '클래식': ['qsP9C-knpGY', 'vSTXLYojkiA', 'MEHF1MU3qyw', 'y_1IOZB-_mI'], '째즈': ['VcIs0Lvbsj4', 'MxWdsVMFm80', 'ENPm0YrHyVc', 'drQY0hZ3fGU'], '런닝맨': ['5yZAUy8cgqs', 'rYTNyMwdlWg', 'c-KLSqKrIf0', 'Oy3vm-DHzeI', 'bVxUuqsP1Qg', 'MY7Q1BHiMZk', '8um_YMOSHqI', 'd2QklvGTJVg', '6e1YGd3NH-w', 'GwtSzxIBnag', 'SaK5gtS8sLg', 'oAgFrhL_YII', 'B7VCUMyTPyc', 'KjPzlC6QE0Y', 'DceGdbWedKk', '61DVaMD7WXQ', 'fvDBOCrOL4E', 'Auw-3xY1waI', 'iXLXLCcGINw'], '1박2일 다시보기': ['10anaq7oTp0'], '무한도전': ['_MnoMA_Y7C4', '-uvhZ7o3x04', 'wFDXalRMpXw', 'A1EmZJDFm6k'], '놀면 뭐하니': ['YAgaicXOjas', 'PtvKVNV-4to'], 'tvN': ['hc5petZeyio', 'wLCmXq2_yRo'], '롤': ['lZX6rR2hxmU', 'X96TunaR3Bo', '87M8if6zrrU', 'sKQIFWIXyMY', 'oaIV0s5DXXI', '0ayv0_ujL0k', '1PqhVpGai1c', 'ErhShv4ELL4', 'w_VbC-krBM0', '1ArtO40izV0', 'W-4f0j-CVf4', 'MugUuG4RkL0', 'cmw0vTjjEtg', 'jTwN9qNJ2Zo', 'CCuCAUMuQKA', 'V_t76T4z3LU', 'ZfITPOIHrH4', 'mMK0Ihar6Ds', 'T8BgIPb67vI', 'ev0bEDV3RCo'], '피파': ['9SBPATOqRp4', 'zqxPHdnRo3M', 'le21IGkvvmU'], 'fps': ['6R_3gwH7rJ4', 'PyI4i_Pcl5Q', 'cCyV2qNQkGs', 'lndIkNjkMq4', 'UaCk6fH8rD8', 'zLnup1gskzU', 'oFNsLSN8rZU', 'SdQL9_zfD0E', '5b1r0Sa58Ko', 'L0L8wg_GKl0', 'OBT53lVZJrY', 'PpNSKdrweJs', 'SI81QuBE28M', 'w5kQU8NvIfk', '09H9O_dnXgY', 'VNtiWnCnvcU', 'lrTp0EbRpxE', 'iJ4juLEh088', 'AM5gEsqIrQk'], '배그': ['mZOGvvG8xCo', '2VdWU9XLFBs', 'Tbv62o0b664', 'cj0Em8Gcioo', 'ZHNTA57Tngo', '-LYzPhPnON4', 'nK81lcIgDWY', 'S8XzepC1JkU', 'Fxnivln-TDM', 'vgbMapTQoD4', 'TExCkw_z2II', '8MoHqR9YTVE', '8CbZN4iFYUQ', 'l5PgztHt0ms', 'ZJB4eK6_8IQ', '25AYCGJZns8', 'o_0qP7Ilcy8', 'hQYIxyhvh-4', 'II5hwXKdpHU', '97n9hruy6hY'], '서든어택': ['nFSzBK0vV74', 'qSUBRcIGkdg', 'oj2orHOag7o', '1Qsa0WwLBr4', 'p3xanvMjssc', 'lKZZjK40tEw', 'rOAxMrUxQSU', 'QfZ0gZ6ccJ0', 'm9VDesk6k_A', '5uFec_F3TvY', '4VUazdjOQZ0', '-4AmtlKYAkA', 'S-Vt_ffFqhM', 'nDGSxrrtuUg', 'luOHdx6loZI', 'ipsBOH_LIk4', 'p7_DOMqh9kQ', 'upEiKDm4x1g', '83-w6J84eWE', 'WIZKq5kRGg0'], '어몽어스': ['O5SnsZDns-c', 'EdpKwOG2X80', 'ywLJOsjckFQ', 'vzw_BAb4etc', 'lvsvSQMHxmg', '_Rrav6oDSXA', 'IqoXggsjFfQ', 'GF_UtlAzbr0', 'zLf0zMjZJTM', 'AA9EDlKhAyE', '0yc2MH7kyZk', 'T15XnTl2skk', '3ZaeB5-mQt4', 'oKNEHc1xiQ8', 'TapEl3bKBu8', 'MQyDvGysSlY', 'nt67Er2a87g', 'n5WOw4Ep4mY', '-PxvczUNEnw', 'WruznbP0DhE'], '바이든': ['s7QhC3exrdo', 'LQElZ4N89Hk', '9-_FOYbW--U', 'MS7zuIWmu0Q', 'vK9HzPtdafw', 'XdTqtNjX4PA', 'ybl4AHLTVIE', '750xpOHhSnI', 'pVTRbZn10_c', 'oJy5dczEmuE', 'lPazD0wp1zs', 'g-KxX5XGu5U', '2WAgoPunn5Q', 'm4axFZeUyD4', 'IkX-8YHofjc', 'BNBM_daPZtI', 'aRju707_ymQ', 'jPGIdCB29HQ', 'WRUUEusffYQ', 'pT18Ld2J-EM'], '트럼프': ['AaP8cpFA6Qg', 'WY3qSffhR6o', 'QX-1ZY3R2YY', 'f_5UCufu9xc', 'e96IeaSYC5I', '9-_FOYbW--U', '376iFtKv-7o', 'z1iXjQDwLxU', '750xpOHhSnI', 'vg_LETfsHu4', 'MS7zuIWmu0Q', '5Cv7jPp2_iM', 'Y3nxMnZ42b8', 'sYd7qtzzyaQ', 'hgY5kndWD1M', 'nH8bFg6ijqE', 'QL6dTGwUkzI', 'LQElZ4N89Hk', 'g-KxX5XGu5U', 'D3OReBCp8eY'], '대통령': ['91xMG7oFE1M', 'f_5UCufu9xc', 'ATkS3GElG5g', 'cS_5ESUcZAA', 'zuU_9FAaaFI', 'YRzSPwV9LIE', 'k2XNiN91J_A', 'Z7zD7ajIUX4', '9_PgIIk6h80', 'jDDkqDMT0Gg', 'zyyCN1uuGeQ', 'iLHWeMLv6-U', 'faCoL2kptPU', 'jKIS0U6pB3k', 'Gu7kxLD_MuM', '_aNq1HxWB1Y', '4A3P9Tgm8wc', 'oWxBSKIm8JM', 'bF2v8vbAG1I', 'yY8bWEqUw78'], '중국': ['tdulZOowSz8', 'td6YWqnbpbA', 'ihCYHrJwpVY', 'bZvPPq3VhcI', '2fWjdD2VCjY', '76e50fnjC3w', 'YPi6ne6D64Q', '3T3R7ItyMow', 'iu9ckQCmU3k', 'iOLB9HFuXcA', 'tZNELPaXF1c', 'h_KV_YR2_OU', 'Gwb-PKAo9io', '2crtBfyGELo', '1WzEFhg7K3k', 'tyfqZg3WJ4g', '-0ydCEcxQ-c', 'MDK5WvVAwgM', 'blU7Mxv8RaY', 'DxakZWazMws'], '백종원': ['qWbHSOplcvY', 'm0eeDwbhzM8', 'kR77WlHRZrs', 'iGhOobaQHso', 'wpUsg7f-F4o', '6e-IbuuD6ZU', 'hc5petZeyio', 'wHuvJKdu2TE', 'udirCMIxs_c', 'ZNOExo_FD3E', 'uF_2nYy74lw', 'fEYJSPSGaLI', '0bnFoRQebq0', 'kfGi3rTct8c', 'Mraq0yBPQXE', 'mwMYLipQluk', '-D1qfgYuBfw', 'GP8_7D5eM6A', 'KhKThbfYoMs'], '자취음식': ['rEnnfMFAijU', 'AcWqE4JC9Pg', 'kmIYGGlua9s', 'F6k4ea_CljM', 'uJq4fSwRbD0', '5j-q3gL1gZE', 'ywuRIlq8dBA', 'ePYy2if0lMY', 'PAh36xoHqg0', 'WvYF1Ax1G7c', 'kR77WlHRZrs', 'deg7pjD0lOs', 'poVEutWwRCo', 'g0rJjcPiWnI', '-4aR8F_T7rU', '_wQvrd4gCiU', 'wevmmDP5jgw', 'Jd9af-re9iE', 'SlRaNQardN8'], '편의점 음식만들기': ['hnvIoHkKKSc', 'TKFpBXl2w-A', 'z6VAEVhluDs', '3iMq4Kdq8cQ', '5AJzx-6sQ5M', 'lgRXx7DJbCY', '-hyL57AtnX8', 'Tcy74iZYNSM', 'U75Zto-R_cY', '6BOOefgnarE', 'Oe2n-UzTuZ0', 'o0QxvVlY7pU', 'g_yyvAKHpsk', 'lv6R8wGjLG4', 'mdv3cIgE2r8', 'BDe5vQfc0iw', 'lO0V19pWE_s', 'OgjUBxTS1FE', 'beZrxBOOWtc', 'xJCoByOe0D8'], '몸에 좋은 음식': ['sO0Vk4v84-Q', 'QASWk6Zslwo', 'ofxYXCQUIZU', '69rSSAW-sGs', 'QBhBvGu0LW0', 'WsinkRH16UU', '6grBm64NUuA', 'fumpEZHiGSs', 'YTGmipgUazY', 'Y_vs2J-t-Kg', '7t6HR9fNkus', 'FU2h84mSLxM', 'Vok2Zqq8SH8', 'xI6Q-I4wxdM', 'WaPeTq1mg-s', 'fxMZRqFU3us', '3zZ_mrXPPNk', 'ru9bXwn6_Eg', 'NzI0TEyZPCA', 'yn2ILPliDfI'], '고양이 영상': ['1R752fMKjvA', 'f2Wcy5mi9Ck', 'I1ms7iiGgYQ', 'lZJFhEStBS8', 't9P4k27dogk', 'nWMMbZG9aKM', 'OWOPZApMOqE', 'rbz5siyFS9s', 'uZFdYhz4ZFU', 'bwQ_jw4lZ3g', 'D99LZeZbdyk', 'QEZzngmPzlQ', 'LGv38Ny4RKc', 'G8Szvw2nGa4', 'RM4rK3vHa_c', '435bSRyWoAo', 'D52kRtlntGI', '2vBU7H6OZgw', 'QRFqphsEZeU', 'J3vLTifqG7U'], '비글': ['J_M-zwOJb2k', '95Tu-WpkGkQ', 'lE2hZXmHGDU', 'GClkqj5JXqc', 'lX7_I5IDQVw', 'K1eWhsxcPX0', 'gjdQkQkX7ks', 'x1TMRJUxxiY', 'Z9sZbnA6EyE', 'Dr-CcGrXb1Y', 'IU9hZ7yhMSM', 'FdP7CGs13I8', '3dRk6ZixmlY', 'gMBadP72Zx0', 'zwXqAo3hmzA', 'ojcj2LmdG3o', 'NvXG08eBtmI', 'oqkB6cVJ_-M', '2kEa4hTSJ7Q', 'iBoH-2-Oi3Q'], '해수어': ['2cxftQrBDHU', 'YxNDWhnzk0E', 'eb2nyvK6h2k', '9G5UqTVA7YI', 'QAhayrF5s1w', 'ecu8Ena_NDQ', 'fbSuRN23Dic', 'pfCbyzyl9B4', 'iP2lKfPJ6qE', 'dn99jOoZNJI', 'lV9hFbmD-PU', 'RieJTVpeEV0', 'MNzB5L60ytA', '73p6P0Xa0vU', 'jYEj3f3nncU', '4S1P0xe9siI', 'lvpQI9fdEEE', 'ZCFTEKR1Ypg', 'Nylo-SpEnFY'], '상어': ['zxWlDeaq2vY', 'cK8SLPmyJkE', 'n_Bs5Oht0ww', 'niqDv_Cho20', '4QNdcz1927s', '9jCOZWu7I-o', 'eKuetncvKLQ', 'k_DrienZyp8', 'Oxmo4Z8ydIQ', 'zcsJQvbgE5E', 's_3vVYHoilI', 'tbJmQ5zYmro', 'Jq2LqzNPhOo', 'n7DGG8_gBKE', 'lpaaUMluenI', 'EbPIhs0tfkU', 'LzLvCAR6M1w', 'ZtU3OU8DuJc', 'LQy6gAogliE', 'IMdowhiALuY']}

In [ ]:
# 특정 검색어 키워드 기반으로 크롤링 해 주는 크롤러 정의
def scrap_search_detail_video_data(video_id_dict):
    # 모든 결과를 누적해서 보여주는 df 정의
#     youtube_raw_data_columns = ['title', 'authorNick', 'description', 'urlPath', 'thumbUrl', 'likeCnt', 'dislikeCnt',
#                                 'followCnt', 'viewCnt', 'duration', 'utbCategory', 'utbCateSN', 'isFamilyFriendly',
#                                 'titleHashTags', 'descHashTags', 'utbVideoID', 'authorID', 'yUploadDateTime', 'keywords']
    
    # 모든 DF를 누적할 DataFrame정의
    # total_df = pd.DataFrame(columns = youtube_raw_data_columns)
    
    # 각 video_id에 해당하는 유튜브 영상 별로 상세 데이터 수집
    for keyword, youtube_id_list in video_id_dict.items():
        
        print("==========outer loop==========")
        print("keyword: ", keyword)
        print("video_id_list: ", youtube_id_list)
        
        # 수집할 데이터의 리스트
        title_list = []
        author_list = []
        description_list = []
        url_list = []
        thumb_url_list = []
        like_cnt_list = []
        dislike_cnt_list = []
        follow_cnt_list = []
        view_cnt_list = []
        duration_list = []
        youtube_category_list = []
        youtube_category_sn_list = []
        is_family_friendly_list = []
        title_hash_tags_list = []
        description_hash_tags_list = []
        video_id_list = [] 
        author_id_list = []
        yupload_time_list = []
        keywords_list = []
        
        # youtube_raw_data를 받을 dictionary 정의
        youtube_raw_data = {}
        
        for video_id in youtube_id_list:
            print("==========inner loop==========")
            
            # webdriver 초기화
            driver_path = './chromedriver'
            driver = webdriver.Chrome(driver_path)
            driver.implicitly_wait(100) # or bigger second

            # origin_url 정의
            origin_url = 'https://www.youtube.com/watch?v='

            # 넣어주는 url 값
            url = origin_url + video_id

            # scriptTag에서 긁어지는 데이터
            script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'

            # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'lxml')

            # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
            yt_initial_data_xpath = '/html/body/script[19]'
            
            driver.get(url)
            time.sleep(5)
            
            # 데이터를 담을 변수 정의(scriptTag, metadata, ytInitialData)
            try:
                # Data 크롤링
                # 각 유형별로 다른 로직으로 데이터를 크롤링한다
                # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
                script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

                script_tag_json = json.loads(script_tag_str)

                time.sleep(5)

                # print("script_tag_json: ", script_tag_json)
                # print("script_tag_json_type: ", type(script_tag_json))

                # Data result test
                title = script_tag_json['name']
                author = script_tag_json['author']
                description = script_tag_json['description']
                thumb_url = script_tag_json['thumbnailUrl'][0]
                view_cnt = int(script_tag_json['interactionCount'])
                duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
                youtube_category = script_tag_json['genre']
                # addition : youtube_category_sn
                youtube_category_sn = youtube_category_dict[youtube_category]

                print()
                print("=" * 20, "Youtube Raw Data Descriptions", "=" * 20)
                print("title: ", title)
                print("author: ", author)
                print("description: ", description)
                print("thumb_url: ", thumb_url)
                print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
                print("duration: ", duration, " type: ", type(duration))
                print("youtube_category: ", youtube_category)
                print("youtube_category_id: ", youtube_category_sn)

                # meta tag data crawling
                is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
                author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
                yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
                keywords = ",".join([tag['content'].replace("#", "") for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

                print("is_family_friendly: ", is_family_friendly)
                print("video_id: ", video_id)
                print("author_id: ", author_id)
                print("yupload_time: ", yupload_time)
                print("keywords: ", keywords)

                # ytInitialData crawling
                yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
                yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
                # print("yt_initial_data: ", yt_initial_data_str)

                time.sleep(5)

                # data to json
                # json loading이 간헐적으로 안되는 현상 수정 필요
                yt_initial_data_json = json.loads(yt_initial_data_str)

                # print("yt_initial_data_json: ", yt_initial_data_json)

                # video_info_renderer
                video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
                video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
                video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

                # print("video_primary_info_renderer: ", video_primary_info_renderer)
                # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

                # 좋아요 & 싫어요
                like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

                print("like_dislike_cnt: ", like_dislike_cnt)

                like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
                dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))

                print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
                print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

                # 제목 타이틀 해시태그
                # 타이틀 해시태그가 없는 경우 빈문자열로 처리
                raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
                if raw_title_hash_tags != "":
                    title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
                    title_hash_tag_list = []
                    for tag in title_hash_tags:
                        if tag['text'] not in [" ", ""]:
                            title_hash_tag_list.append(tag['text'][1:])
                            # print("title_hash_tag: ", tag['text'][1:])
                    title_hash_tags_str = ",".join(title_hash_tag_list)
                else:
                    title_hash_tags_str = ""
                print("title_hash_tags_str: ", title_hash_tags_str)

                # 내용 해시태그
                description_hash_tags = video_secondary_info_renderer['description']['runs']
                description_hash_tag_list = []
                for tag in description_hash_tags:
                    if tag['text'] not in [" ", ""]:
                        if "#" in tag['text']:
                            description_hash_tag_list.append(tag['text'][1:])
                            # print("description_hash_tag: ", tag['text'][1:])

                description_hash_tags_str = ",".join(description_hash_tag_list)
                print("description_hash_tags_str: ", description_hash_tags_str)

                # 구독자 수
                # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
                raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
                if raw_follow != "":
                    raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
                    follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
                    print("follow_cnt: ", follow_cnt)

                    # 구독자 수 수치화 필터링
                    num_count_dict = {
                        '천' : 1000,
                        '만' : 10000,
                        '억' : 100000000,
                        '조' : 1000000000000
                    }

                    # print("follow_cnt: ", follow_cnt)
                    # print("follow_num_cnt: ", follow_cnt[-1])
                    # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
                    if follow_cnt[-1] in num_count_dict.keys():
                        follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
                    else:
                        follow_cnt_num = int(follow_cnt)
                else:
                    follow_cnt_num = 0

                print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))

                # 예외처리에 걸리지 않고 무사히 넘어왔다면 list에 데이터가 담긴다
                # scriptTag data appending
                title_list.append(title)
                author_list.append(author)
                description_list.append(description)
                url_list.append(url)
                thumb_url_list.append(thumb_url)
                view_cnt_list.append(view_cnt)
                duration_list.append(duration)
                youtube_category_list.append(youtube_category)
                youtube_category_sn_list.append(youtube_category_sn)

                # meta tag data appending
                is_family_friendly_list.append(is_family_friendly)
                video_id_list.append(video_id)
                author_id_list.append(author_id)
                yupload_time_list.append(yupload_time)
                keywords_list.append(keywords)

                # yInitialData data appending
                like_cnt_list.append(like_cnt)
                dislike_cnt_list.append(dislike_cnt)
                title_hash_tags_list.append(title_hash_tags_str)
                description_hash_tags_list.append(description_hash_tags_str)
                follow_cnt_list.append(follow_cnt_num)
                
            except Exception as e:
                print()
                print("Exception: ", e)
                driver.close()
                continue
            
            print("=" * 20, "Youtube Raw Data Descriptions Ended", "=" * 20, "\n")
            
            driver.close()

        # 얻어온 데이터를 dictionary에 저장
        youtube_raw_data['title'] = title_list
        youtube_raw_data['authorNick'] = author_list
        youtube_raw_data['description'] = description_list
        youtube_raw_data['urlPath'] = url_list
        youtube_raw_data['thumbUrl'] = thumb_url_list
        youtube_raw_data['likeCnt'] = like_cnt_list
        youtube_raw_data['dislikeCnt'] = dislike_cnt_list
        youtube_raw_data['followCnt'] = follow_cnt_list
        youtube_raw_data['viewCnt'] = view_cnt_list
        youtube_raw_data['duration'] = duration_list
        youtube_raw_data['utbCategory'] = youtube_category_list
        youtube_raw_data['utbCateSN'] = youtube_category_sn_list
        youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
        youtube_raw_data['titleHashTags'] = title_hash_tags_list
        youtube_raw_data['descHashTags'] = description_hash_tags_list
        youtube_raw_data['utbVideoID'] = video_id_list
        youtube_raw_data['authorID'] = author_id_list
        youtube_raw_data['yUploadDateTime'] = yupload_time_list
        youtube_raw_data['keywords'] = keywords_list
        
        print("youtube_raw_data: ", youtube_raw_data)
        
        # DataFrame 생성
        df = pd.DataFrame(youtube_raw_data)
        
        total_df.append(df, ignore_index = True)

        # 모아둔 DataFrame을 csv로 변환
        df.to_csv("./"+keyword+"_youtube_raw_data.csv", sep="|", index=False, encoding='utf-8-sig')

    return total_df

In [ ]:
scrap_search_detail_video_data(youtube_id_dict)

In [ ]:
# csv file 불필요한 0번째 컬럼 제거(test)
df = pd.read_csv("./공룡_youtube_raw_data2.csv", index_col=False)
df = df.drop("Unnamed: 0", axis=1)

df.to_csv("./공룡_youtube_raw_data2"+"_fix"+".csv", index=False, encoding='utf-8-sig')

In [ ]:
# 기존에 default index_column을 추가한 csv 파일을 불러온 후 index_column 제거 후 다시 저장
for keyword in all_search_keyword_list:
    df = pd.read_csv("./"+keyword+"_youtube_raw_data.csv", index_col=False)
    df = df.drop("Unnamed: 0", axis=1)
    
    df.to_csv("./"+keyword+"_youtube_raw_data"+"_fix"+".csv", index=False, encoding='utf-8-sig')

In [ ]:
# 기존 데이터에서 문자열 데이터 중 "," -> "|"로 변경 및 문자열 내부의 \"를 ""로 변경
df = pd.read_csv("./공룡_youtube_raw_data2"+"_fix"+".csv", index_col=False, encoding='utf-8-sig')

# 문자열 정리(filtering)
# 문자열을 필터링하는 함수 정의
def filter_string_data(data):
    data = data.replace(",", "|").replace('"', '')
    
    return data

df['title'] = df['title'].apply(filter_string_data)
df['authorNick'] = df['authorNick'].apply(filter_string_data)
df['description'] = df['description'].apply(filter_string_data)

# df
df.to_csv("./공룡_youtube_raw_data2"+"_fix2"+".csv", sep="|", index=False, encoding='utf-8-sig')

In [ ]:
# description 제거 작업
df3 = pd.read_csv('./공룡_youtube_raw_data2_fix2(1).csv', sep="|", index_col=False, encoding='utf-8-sig')
df3

# removing description column
modified_list = ['title', 'authorNick', 'urlPath', 'thumbUrl', 'likeCnt', 'dislikeCnt', 
                 'followCnt', 'viewCnt', 'duration', 'utbCategory', 'utbCateSN', 'isFamilyFriendly', 
                 'titleHashTags', 'descHashTags', 'utbVideoID', 'authorID', 'yUploadDateTime', 'keywords']

df3_mod = df3[modified_list]
df3_mod.to_csv("./공룡_youtube_raw_data2"+"_fix3"+".csv", sep="|", index=False, encoding='utf-8-sig')

In [ ]:
# 기존 데이터에서 문자열 데이터 중 "," -> "|"로 변경 및 문자열 내부의 \"를 ""로 변경
# 임시방편으로 description 삭제 진행
# 문자열 정리(filtering)
# 문자열을 필터링하는 함수 정의
def filter_string_data(data):
    data = data.replace(",", "|").replace('"', '')

    return data

# removing description column
modified_list = ['title', 'authorNick', 'urlPath', 'thumbUrl', 'likeCnt', 'dislikeCnt', 
                 'followCnt', 'viewCnt', 'duration', 'utbCategory', 'utbCateSN', 'isFamilyFriendly', 
                 'titleHashTags', 'descHashTags', 'utbVideoID', 'authorID', 'yUploadDateTime', 'keywords']

for keyword in all_search_keyword_list:
    df = pd.read_csv("./"+keyword+"_youtube_raw_data"+"_fix"+".csv", index_col=False, encoding='utf-8-sig')

    df['title'] = df['title'].apply(filter_string_data)
    df['authorNick'] = df['authorNick'].apply(filter_string_data)
    
    df = df[modified_list]

    # df
    df.to_csv("./"+keyword+"_youtube_raw_data"+"_fix2"+".csv", sep="|", index=False, encoding='utf-8-sig')

In [ ]:
# 데이터 단일 크롤링 테스트
def scrap_test_detail_video_data(video_id='y57Zldv2bik'):
    
    # webdriver 초기화
    driver_path = './chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # origin_url 정의
    origin_url = 'https://www.youtube.com/watch?v='
    
    # 테스트용 단일 youtube_id 정의
    # (임시) 딘일로 넣어주는 값
    # video_id = 'y57Zldv2bik'
    
    # youtube_detail_raw_data를 받을 dictionary 정의
    youtube_raw_data = {}
    
    # 넣어주는 url 값
    url = origin_url + video_id
    
    # scriptTag에서 긁어지는 데이터
    script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'
    
    # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'lxml')
    
    # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
    yt_initial_data_xpath = '/html/body/script[19]'
    
    # 수집할 데이터의 리스트
    title_list = []
    author_list = []
    description_list = []
    url_list = []
    thumb_url_list = []
    like_cnt_list = []
    dislike_cnt_list = []
    follow_cnt_list = []
    view_cnt_list = []
    duration_list = []
    youtube_category_list = []
    youtube_category_sn_list = []
    is_family_friendly_list = []
    title_hash_tags_list = []
    description_hash_tags_list = []
    video_id_list = [] 
    author_id_list = []
    yupload_time_list = []
    keywords_list = []
    
    driver.get(url)
    time.sleep(5)
    
    # Data 크롤링
    # 각 유형별로 다른 로직으로 데이터를 크롤링한다
    
    # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
    script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

    script_tag_json = json.loads(script_tag_str)
    
    time.sleep(5)

    # print("script_tag_json: ", script_tag_json)
    # print("script_tag_json_type: ", type(script_tag_json))

    # Data result test
    title = script_tag_json['name']
    author = script_tag_json['author']
    description = script_tag_json['description']
    thumb_url = script_tag_json['thumbnailUrl'][0]
    view_cnt = int(script_tag_json['interactionCount'])
    duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
    youtube_category = script_tag_json['genre']
    # addition : youtube_category_sn
    youtube_category_sn = youtube_category_dict[youtube_category]

    print("title: ", title)
    print("author: ", author)
    print("description: ", description)
    print("thumb_url: ", thumb_url)
    print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
    print("duration: ", duration, " type: ", type(duration))
    print("youtube_category: ", youtube_category)
    print("youtube_category_id: ", youtube_category_sn)
    
    # scriptTag data appending
    title_list.append(title)
    author_list.append(author)
    description_list.append(description)
    url_list.append(url)
    thumb_url_list.append(thumb_url)
    view_cnt_list.append(view_cnt)
    duration_list.append(duration)
    youtube_category_list.append(youtube_category)
    youtube_category_sn_list.append(youtube_category_sn)
    
    
    # meta tag data crawling
    is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
    author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
    yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
    keywords = ",".join([tag['content'] for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

    print("is_family_friendly: ", is_family_friendly)
    print("video_id: ", video_id)
    print("author_id: ", author_id)
    print("yupload_time: ", yupload_time)
    print("keywords: ", keywords)
    
    # meta tag data appending
    is_family_friendly_list.append(is_family_friendly)
    video_id_list.append(video_id)
    author_id_list.append(author_id)
    yupload_time_list.append(yupload_time)
    keywords_list.append(keywords)
    
    # ytInitialData crawling
    yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
    yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
    # print("yt_initial_data: ", yt_initial_data_str)

    time.sleep(5)

    # data to json
    # json loading이 간헐적으로 안되는 현상 수정 필요
    yt_initial_data_json = json.loads(yt_initial_data_str)

    # print("yt_initial_data_json: ", yt_initial_data_json)

    # video_info_renderer
    video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
    video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
    video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

    # print("video_primary_info_renderer: ", video_primary_info_renderer)
    # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

    # 좋아요 & 싫어요
    like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

    print("like_dislike_cnt: ", like_dislike_cnt)
    
    like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
    dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))
    
    print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
    print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

    # 제목 타이틀 해시태그
    raw_title_hash_tags = video_primary_info_renderer.get('superTitleLink', "")
    if raw_title_hash_tags != "":
        title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
        title_hash_tag_list = []
        for tag in title_hash_tags:
            if tag['text'] not in [" ", ""]:
                title_hash_tag_list.append(tag['text'][1:])
                # print("title_hash_tag: ", tag['text'][1:])
        title_hash_tags_str = ",".join(title_hash_tag_list)
    else:
        title_hash_tags_str = raw_title_hash_tags
    print("title_hash_tags_str: ", title_hash_tags_str)

    # 내용 해시태그
    description_hash_tags = video_secondary_info_renderer['description']['runs']
    description_hash_tag_list = []
    for tag in description_hash_tags:
        if tag['text'] not in [" ", ""]:
            if "#" in tag['text']:
                description_hash_tag_list.append(tag['text'][1:])
                # print("description_hash_tag: ", tag['text'][1:])

    description_hash_tags_str = ",".join(description_hash_tag_list)
    print("description_hash_tags_str: ", description_hash_tags_str)

    # 구독자 수
    # 구독자 수가 없는 경우에 대한 예외처리(KeyError) -> 0 으로 처리
    raw_follow = video_secondary_info_renderer['owner']['videoOwnerRenderer'].get('subscriberCountText', "")
    if raw_follow != "":
        raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
        follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
        print("follow_cnt: ", follow_cnt)

        # 구독자 수 수치화 필터링
        num_count_dict = {
            '천' : 1000,
            '만' : 10000,
            '억' : 100000000,
            '조' : 1000000000000
        }

        # print("follow_cnt: ", follow_cnt)
        # print("follow_num_cnt: ", follow_cnt[-1])
        # 천, 만, 억, 조 에 걸리는 경우 -> 그 만큼 곱해서 리턴, 그게 아닐 경우 그냥 정수화 후 리턴
        if follow_cnt[-1] in num_count_dict.keys():
            follow_cnt_num = int(float(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]])
        else:
            follow_cnt_num = int(follow_cnt)
    else:
        follow_cnt_num = 0
    print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))
    
    # yInitialData data appending
    like_cnt_list.append(like_cnt)
    dislike_cnt_list.append(dislike_cnt)
    title_hash_tags_list.append(title_hash_tags_str)
    description_hash_tags_list.append(description_hash_tags_str)
    follow_cnt_list.append(follow_cnt_num)
    
    driver.close()
    
    # 얻어온 데이터를 dictionary에 저장
    youtube_raw_data['title'] = title_list
    youtube_raw_data['authorNick'] = author_list
    youtube_raw_data['description'] = description_list
    youtube_raw_data['urlPath'] = url_list
    youtube_raw_data['thumbUrl'] = thumb_url_list
    youtube_raw_data['likeCnt'] = like_cnt_list
    youtube_raw_data['dislikeCnt'] = dislike_cnt_list
    youtube_raw_data['followCnt'] = follow_cnt_list
    youtube_raw_data['viewCnt'] = view_cnt_list
    youtube_raw_data['duration'] = duration_list
    youtube_raw_data['utbCategory'] = youtube_category_list
    youtube_raw_data['utbCateSN'] = youtube_category_sn_list
    youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
    youtube_raw_data['titleHashTags'] = title_hash_tags_list
    youtube_raw_data['descHashTags'] = description_hash_tags_list
    youtube_raw_data['utbVideoID'] = video_id_list
    youtube_raw_data['authorID'] = author_id_list
    youtube_raw_data['yUploadDateTime'] = yupload_time_list
    youtube_raw_data['keywords'] = keywords_list
    
    # DataFrame 생성
    df = pd.DataFrame(youtube_raw_data)
    
    return df

In [ ]:
# 단일 크롤링 테스트 결과
# scrap_test_detail_video_data(video_id='VjwJSj0AW4g')

In [ ]:
# scrap_test_detail_video_data(video_id='doorlUyE9pI')

In [ ]:
# scrap_test_detail_video_data(video_id='L0Y38pQbEeQ')

In [ ]:
# scrap_test_detail_video_data(video_id='7eYu507yrQ8')